In [ ]:
# !pip install langchain-text-splitters qdrant-client langchain-qdrant sentence-transformers torch torchvision torchaudio
# !pip install ragas rapidfuzz ipywidgets langchain-huggingface accelerate

#!pip install ragas datasets pandas openai
#!pip install langchain langchain-openai langchain-community

In [1]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import torch,uuid
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFaceEmbeddings ,HuggingFacePipeline
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.language_models.llms import LLM
from typing import Any, List, Optional

## ragas
from datasets import Dataset
from ragas import evaluate
from ragas.metrics.collections import Faithfulness, AnswerRelevancy
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:
### 파일 data load

# 파일 경로 지정
file_path = '사회복지_법령_전체.txt'

# 파일 내용이 담긴 변수
law_data=''


# 파일 내용 load
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        law_data = f.read()
    print(f"전체 글자 수: {len(law_data):,}자")
except FileNotFoundError:
    print("파일을 찾을 수 없습니다.")

전체 글자 수: 4,987,798자


In [3]:
### 문서 분할

# 필요없는 부분 법령명으로 replace
law_data = law_data.replace("""판례
연혁
위임행정규칙
규제
생활법령
한눈보기""","")

# 분할할 방식 설정
text_splitter = CharacterTextSplitter(
    separator="\n\n\n",
    chunk_size=1,           # 구분자 기준으로 바로 쪼개지도록 최소값 설정
    chunk_overlap=0,        # 중복 없음
    is_separator_regex=False # 일반 문자열로 취급
)

# 분할
chunks = text_splitter.split_text(law_data)

# vectorDB에 넣을 문서 리스트
documents=[]

# vectorDB에 넣을 형식으로 변환
for chunk in chunks:
    #문서의 법령을 제목으로 사용하기 위한 개행으로 split
    law_name = chunk.splitlines()
    
    # vectorDB에 넣을 형식으로 변환
    doc = Document(
        page_content=chunk,
        metadata={
            "law_name": law_name[0], 
            "length": len(chunk)
        }
    )

    # vectorDB에 넣을 list에 변환한 문서 append
    documents.append(doc)


Created a chunk of size 1764, which is longer than the specified 1
Created a chunk of size 329, which is longer than the specified 1
Created a chunk of size 1064, which is longer than the specified 1
Created a chunk of size 1089, which is longer than the specified 1
Created a chunk of size 1017, which is longer than the specified 1
Created a chunk of size 2752, which is longer than the specified 1
Created a chunk of size 2117, which is longer than the specified 1
Created a chunk of size 449, which is longer than the specified 1
Created a chunk of size 459, which is longer than the specified 1
Created a chunk of size 2846, which is longer than the specified 1
Created a chunk of size 1244, which is longer than the specified 1
Created a chunk of size 1147, which is longer than the specified 1
Created a chunk of size 3147, which is longer than the specified 1
Created a chunk of size 4551, which is longer than the specified 1
Created a chunk of size 8678, which is longer than the specified 

In [4]:
# huggingface login
import os 
from dotenv import load_dotenv
from huggingface_hub import login

load_dotenv()
login(os.getenv('HUGGINGFACE_API_KEY'))

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
### VectorDB에 저장

# 모델에 따라 달라질 코드(임베딩)
embeddings = HuggingFaceEmbeddings(
    model_name="woong0322/ko-legal-sbert-finetuned",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True} # 의미 기반 검색 최적화
)

# qdrant 연결
url = "http://localhost:6333"
client = QdrantClient(url=url)

# 각 법령의 구분 키값
ids = [
    str(uuid.uuid5(uuid.NAMESPACE_DNS, f"{doc.metadata['law_name']}_{i}")) 
    for i, doc in enumerate(documents)
]

# DB명
collection_name = "B-TEAM"

# vectorDB에 저장
vector_store = QdrantVectorStore.from_documents(
    documents=documents,
    embedding=embeddings,
    ids = ids,
    url=url,
    collection_name=collection_name
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

c:\Users\Playdata\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Playdata\.cache\huggingface\hub\models--woong0322--ko-legal-sbert-finetuned. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [7]:
### RAG

# 임베딩 모델 설정
embeddings = HuggingFaceEmbeddings(
    model_name="woong0322/ko-legal-sbert-finetuned",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# vectorDB연결
url = "http://localhost:6333"
collection_name = "B-TEAM"

vector_store = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name=collection_name,
    url=url,
)

# 질문에 대한 답은 가장 유사한 것 하나
retriever = vector_store.as_retriever(search_kwargs={"k": 1})

#llm 모델 설정
model_id = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)

torch_dtype = torch.float16 if device == "cuda" else torch.float32
model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    device_map="auto",               
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    trust_remote_code=True
)

model_engine = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    do_sample=True,
    return_full_text=False,
)

llm = HuggingFacePipeline(pipeline=model_engine)

# 프롬프트 구성
qa_system_prompt = """너는 대한민국 사회복지사를 지원하는 법령 검색 전문 AI 도우미이다.

**기본 규칙**
답변은 항상 한국어로 한다.
당신은 전문가 도우미로 제공된 [법령 정보] 데이터에 기반한 정보를 제공한다.
질문이 불확실한 경우 한 번 더 질문하여 질문을 구체화 한다.
정보가 부족하면 "정보를 찾을 수 없다"고 답한다.
데이터에 없는 내용은 추측해 답하지 않는다.

역할:
- 사회복지 관련 법령(예: 사회복지사업법, 노인복지법, 아동복지법, 장애인복지법 등)을 근거 문서에 기반하여 정확하게 안내한다.
- 사용자의 질문에 대해 관련 법 조항을 우선적으로 제시하고, 사회복지 실무 관점에서 이해하기 쉽게 설명한다.

원칙:
- 제공된 문서[법령 정보] 안의 정보만을 근거로 답변한다.
- 문서에 없는 내용은 추측하지 말고 "관련 근거를 찾을 수 없다"고 답변한다.
- 법률적 최종 판단이나 자문은 하지 않는다.
- 항상 조항 번호와 법령명을 명시한다.


**질문 처리 절차 **
1. 질문에서 "핵심단어"를 인식한다.
-"핵심단어"란 질문자가 알고 싶어하는 정보를 찾기 위한 keyword 다.
-예시: "사회복지법인 설립 조건을 알고 싶어", "사회복지법인을 만들려면 어떻게 해야하지?" 등의 질문의 "핵심단어"는 "사회복지법인", "설립 조건", "만들다"이다.

2. "핵심단어"를 기준으로 조회한 법령들 중에서 질문과 가장 유사한 법령을 찾는다.
-필요한 경우 질문과 연관된 추가 조항도 검토하여 답변의 완성도를 높인다.
-예시: "사회복지법인 설립 조건을 알고 싶어" -> 제16조(법인의 설립허가) 항목 이외에 17조(정관)등에 대한 내용까지 요약 정리.


답변 형식:
1. 물어본 질문에 대해 간결하게 답변할 것.
2. 근거가 된 [법령정보]에 대해 3가지 이하로 첨부할 것. 최소한으로 덧붙인다.
3. 주의사항 또는 한계 안내할 것.

사용자 : (사용자의 질문 내용)
모델 : (질문에 대해 대화하듯이 친절하게 설명)

관련 조항
1. 제O조(명칭) : 조항의 핵심 내용 
2. 제O조(명칭) : 조항의 핵심 내용 
3. 제O조(명칭) : 조항의 핵심 내용 

주의사항 또는 한계
이 답변은 법률 자문이 아니며, 구체적인 행정 해석이나 적용 여부는 관할 행정기관 또는 법률 전문가에게 확인해야 한다.

[법령 정보]:
{context}"""

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", qa_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

# LCEL 체인
def extract_content(docs):
    return docs[0].page_content if docs else "관련 법령 없음"

# LCEL 체인: 구조 변경 없이 그대로 사용
rag_chain = (
    {
        "context": (lambda x: x["input"]) | retriever | extract_content,
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"]
    }
    | qa_prompt 
    | llm  
)


configuration_exaone.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- configuration_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`torch_dtype` is deprecated! Use `dtype` instead!


modeling_exaone.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct:
- modeling_exaone.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:

# 성능 평가 모델
evaluator_llm = ChatOpenAI(model="gpt-4o-mini") # 판사 LLM
evaluator_embeddings = OpenAIEmbeddings()

#실제 대화
chat_history = []
print("법률 상담을 시작합니다. (종료: exit)")

while True:
    user_input = input("\n나: ")
    if user_input.lower() in ["exit", "종료"]: break

    #성능 평가 텍스트
    retrieved_docs = retriever.invoke(f"query: {user_input}")
    contexts = [doc.page_content for doc in retrieved_docs]

    # 체인 호출
    response = rag_chain.invoke({"input": user_input, "chat_history": chat_history})
    print(f"{response}")

    #성능 평가
    current_eval_data = {
        "user_input": [user_input],
        "response": [str(response)],
        "retrieved_contexts": [contexts]
    }
    eval_dataset = Dataset.from_dict(current_eval_data)

    score = evaluate(
            dataset=eval_dataset,
            metrics = [
                Faithfulness(llm=evaluator_llm),
                AnswerRelevancy(
                    llm=evaluator_llm,
                    embeddings=evaluator_embeddings
                ),
            ],
            llm=evaluator_llm,
            embeddings=evaluator_embeddings
        )
        
        # 점수 출력
    f_score = score["faithfulness"]
    ar_score = score["answer_relevancy"]
    print(f"   [ 성능 점수] 충실도(Faithfulness): {f_score:.2f} | 관련성(Relevancy): {ar_score:.2f}")

    # 대화 기록 업데이트 (최근 3턴만 유지하여 CPU 부담 감소)
    chat_history.extend([HumanMessage(content=user_input), AIMessage(content=response)])
    chat_history = chat_history[-6:]